# Migrate MongoDB database from `nmdc-schema` `v10.3.0` to `v10.4.0`

There is a migrator for the schema changes from `v10.3.0` to `v10.4.0`. That migrator was added to the `nmdc-schema` repo "late" (i.e. in version `v10.5.4`). There is no migrator for the schema changes, if any, from `v10.4.0` to `10.5.6`. 

## Prerequisites

### 1. Determine MongoDB collections involved.

To determine this, we look at the migrator, itself. We make note of which collections are referenced by that migrator (whether for reading or for writing) and add them to the `COLLECTION_NAMES` list below.

```py
# TODO: Consider separating them into two lists: `COLLECTIONS_TO_DUMP` and `COLLECTIONS_TO_RESTORE`.
```

In [ ]:
# Note: `*arr` in Python is like `...arr` in JavaScript (it's a "spread" operator).
COLLECTION_NAMES: list[str] = [
    "nom_analysis_activity_set",
    "data_object_set",
]
print(str(len(COLLECTION_NAMES)) + " collection names")

In [ ]:
# Eliminate duplicates.
COLLECTION_NAMES = list(set(COLLECTION_NAMES))
print(str(len(COLLECTION_NAMES)) + " collection names (distinct)")

### 2. Coordinate with stakeholders.

We will be enacting full Runtime and Database downtime for this migration. Ensure stakeholders are aware of that.

### 3. Set up environment.

Here, you'll prepare an environment for running this notebook.

1. Start a **MongoDB server** on your local machine (and ensure it does **not** already contain a database named `nmdc`).
    1. You can start a [Docker](https://hub.docker.com/_/mongo)-based MongoDB server at `localhost:27055` by running this command (this MongoDB server will be accessible without a username or password).
       ```shell
       docker run --rm --detach --name mongo-migration-transformer -p 27055:27017 mongo:6.0.4
       ```
2. Create and populate a **notebook configuration file** named `.notebook.env`.
    1. You can use `.notebook.env.example` as a template:
       ```shell
       $ cp .notebook.env.example .notebook.env
       ```
3. Create and populate the two **MongoDB configuration files** that this notebook will use to connect to the "origin" and "transformer" MongoDB servers. The "origin" MongoDB server is the one that contains the database you want to migrate; and the "transformer" MongoDB server is the one you want to use to perform the data transformations. In practice, the "origin" MongoDB server is typically a remote server, and the "transformer" MongoDB server is typically a local server.
    1. You can use `.mongo.yaml.example` as a template:
       ```shell
       $ cp .mongo.yaml.example .mongo.origin.yaml
       $ cp .mongo.yaml.example .mongo.transformer.yaml
       ```
       > When populating the file for the origin MongoDB server, use credentials that have **both read and write access** to the `nmdc` database.

- TODO: Be more specific about the Mongo privileges necessary to perform a `mongodump` and a `mongorestore` that may involve creating/deleting collections.

## Procedure

### Install Python dependencies

In this step, you'll [install](https://saturncloud.io/blog/what-is-the-difference-between-and-in-jupyter-notebooks/) the Python packages upon which this notebook depends.

> Note: If the output of this cell says "Note: you may need to restart the kernel to use updated packages", restart the kernel (not the notebook cells) now.

References: 
- https://pypi.org/project/nmdc-schema/

In [ ]:
%pip install --upgrade pip
%pip install -r requirements.txt
%pip install nmdc-schema==10.5.6  # note: the 10.3.0-to-10.4.0 migrator was introduced in schema package version 10.5.4 and a bugfix was introduced in 10.5.6

### Import Python dependencies

Import the Python objects upon which this notebook depends.

In [ ]:
# Third-party packages:
import pymongo
from jsonschema import Draft7Validator
from nmdc_schema.nmdc_data import get_nmdc_jsonschema_dict
from nmdc_schema.migrators.adapters.mongo_adapter import MongoAdapter
from nmdc_schema.migrators.migrator_from_10_3_0_to_10_4_0 import Migrator

# First-party packages:
from helpers import Config
from bookkeeper import Bookkeeper, MigrationEvent

### Parse configuration files

Parse the notebook and Mongo configuration files.

In [ ]:
cfg = Config()

# Define some aliases we can use to make the shell commands in this notebook easier to read.
mongodump = cfg.mongodump_path
mongorestore = cfg.mongorestore_path

# Perform a sanity test of the application paths.
!{mongodump} --version
!{mongorestore} --version

### Create MongoDB clients

Create MongoDB clients you can use to access the "origin" and "transformer" MongoDB servers.

In [ ]:
# Mongo client for "origin" MongoDB server.
origin_mongo_client = pymongo.MongoClient(host=cfg.origin_mongo_server_uri, directConnection=True)

# Mongo client for "transformer" MongoDB server.
transformer_mongo_client = pymongo.MongoClient(host=cfg.transformer_mongo_server_uri)

# Perform sanity tests of those MongoDB clients' abilities to access their respective MongoDB servers.
with pymongo.timeout(3):
    # Display the MongoDB server version (running on the "origin" Mongo server).
    print("Origin Mongo server version:      " + origin_mongo_client.server_info()["version"])

    # Sanity test: Ensure the origin database exists.
    assert "nmdc" in origin_mongo_client.list_database_names(), "Origin database does not exist."

    # Display the MongoDB server version (running on the "transformer" Mongo server).
    print("Transformer Mongo server version: " + transformer_mongo_client.server_info()["version"])

    # Sanity test: Ensure the transformation database does not exist.
    assert "nmdc" not in transformer_mongo_client.list_database_names(), "Transformation database already exists."

### Create JSON Schema validator

In this step, you'll create a JSON Schema validator for the NMDC Schema.

- TODO: Consider whether the JSON Schema validator version is consistent with the JSON Schema version (e.g. draft 7 versus draft 2019).

In [ ]:
nmdc_jsonschema: dict = get_nmdc_jsonschema_dict()
nmdc_jsonschema_validator = Draft7Validator(nmdc_jsonschema)

# Perform sanity tests of the NMDC Schema dictionary and the JSON Schema validator.
# Reference: https://python-jsonschema.readthedocs.io/en/latest/api/jsonschema/protocols/#jsonschema.protocols.Validator.check_schema
print("NMDC Schema title:   " + nmdc_jsonschema["title"])
print("NMDC Schema version: " + nmdc_jsonschema["version"])

nmdc_jsonschema_validator.check_schema(nmdc_jsonschema)  # raises exception if schema is invalid

### TODO: Revoke write access to the "origin" MongoDB server

This is so people don't make changes to the original data while the migration is happening, given that the migration ends with an overwriting of the original data.

Note: The migrator Mongo user may need additional permissions in order to manipulate Mongo user roles to the extent necessary to accomplish this step.

### Dump collections from the "origin" MongoDB server

Use `mongodump` to dump the collections involved in this migration **from** the "origin" MongoDB server **into** a local directory.

> Since `mongodump` doesn't provide a CLI option we can use to specify the collections we _want_ the dump to include, we use multiple occurrences of the `--excludeCollection` CLI option to exclude each collection we do _not_ want the dump to include. The end result is the same—there's just that extra step involved.

- TODO: Consider ensuring that the local dump target folder is empty before doing this dump.

In [ ]:
# Build a string containing zero or more `--excludeCollection="..."` options, which can be included in a `mongodump` command.
all_collection_names: list[str] = origin_mongo_client["nmdc"].list_collection_names()
non_agenda_collection_names = [name for name in all_collection_names if name not in COLLECTION_NAMES]
exclusion_options = [f"--excludeCollection='{name}'" for name in non_agenda_collection_names]
exclusion_options_str = " ".join(exclusion_options)  # separates each option with a space
print(exclusion_options_str)

In [ ]:
# Dump the not-excluded collections from the "origin" database.
!{mongodump} \
  --config="{cfg.origin_mongo_config_file_path}" \
  --db="nmdc" \
  --gzip \
  --out="{cfg.origin_dump_folder_path}" \
  {exclusion_options_str}

### Load the dumped collections into the "transformer" MongoDB server

Use `mongorestore` to load the dumped collections **from** the local directory **into** the "transformer" MongoDB server.

> Since it's possible that the dump included extra collections (due to someone having created a collection between the time you generated the `--excludeCollection` CLI options and the time you ran `mongodump` above), we will use the `--nsInclude` CLI option to indicate which specific collections—from the dump—we want to load into the "transformer" database.

In [ ]:
# Build a string containing zero or more `--nsInclude="..."` options, which can be included in a `mongorestore` command.
inclusion_options = [f"--nsInclude='nmdc.{name}'" for name in COLLECTION_NAMES]
inclusion_options_str = " ".join(inclusion_options)  # separates each option with a space
print(inclusion_options_str)

In [ ]:
# Restore the dumped collections to the "transformer" MongoDB server.
!{mongorestore} \
  --config="{cfg.transformer_mongo_config_file_path}" \
  --gzip \
  --drop \
  --preserveUUID \
  --stopOnError \
  --dir="{cfg.origin_dump_folder_path}" \
  {inclusion_options_str}

### Transform the collections within the "transformer" MongoDB server

Use the migrator to transform the collections in the "transformer" database.

> Reminder: The database transformation functions are defined in the `nmdc-schema` Python package installed earlier.

> Reminder: The "origin" database is **not** affected by this step.

In [ ]:
import logging

# Setup a logger that writes to a file.
# TODO: Move this logger stuff to `helpers.py`.`
LOG_FILE_PATH = "./tmp.log"
logger = logging.getLogger(name="migrator_logger")
logger.setLevel(logging.DEBUG)
file_handler = logging.FileHandler(LOG_FILE_PATH)
formatter = logging.Formatter(fmt="[%(asctime)s\t%(name)s\t%(levelname)s]\t%(message)s")
file_handler.setFormatter(formatter)
if logger.hasHandlers():
    logger.handlers.clear()  # avoid duplicate log entries
logger.addHandler(file_handler)

In [ ]:
# Instantiate a MongoAdapter bound to the "transformer" database.
adapter = MongoAdapter(
    database=transformer_mongo_client["nmdc"],
    on_collection_created=lambda name: print(f'Created collection "{name}"'),
    on_collection_renamed=lambda old_name, name: print(f'Renamed collection "{old_name}" to "{name}"'),
    on_collection_deleted=lambda name: print(f'Deleted collection "{name}"'),
)

# Instantiate a Migrator bound to that adapter.
migrator = Migrator(adapter=adapter, logger=logger)

# Execute the Migrator's `upgrade` method to perform the migration.
migrator.upgrade()

### Validate the transformed documents

Now that we have transformed the database, validate each document in each collection in the "transformer" MongoDB server.

> Reference: https://github.com/microbiomedata/nmdc-runtime/blob/main/metadata-translation/src/bin/validate_json.py

In [ ]:
# Ensure that, if the (large) "functional_annotation_agg" collection is present in `COLLECTION_NAMES`,
# it goes at the end of the list we process. That way, we can find out about validation errors in
# other collections without having to wait for that (large) collection to be validated before them.
ordered_collection_names = sorted(COLLECTION_NAMES.copy())
large_collection_name = "functional_annotation_agg"
if large_collection_name in ordered_collection_names:
    ordered_collection_names = list(filter(lambda n: n != large_collection_name, ordered_collection_names))
    ordered_collection_names.append(large_collection_name)

# TODO: Only validate documents in the collections that we will be restoring.
#       Note: If a collection listed in `COLLECTION_NAMES` doesn't exist in the transformation
#             database anymore, the inner `for` loop will just have zero iterations.
for collection_name in ordered_collection_names:

    collection = transformer_mongo_client["nmdc"][collection_name]
    num_documents_in_collection = collection.count_documents({})
    print(f"Validating collection {collection_name} ({num_documents_in_collection} documents)")

    for document in collection.find():
        # Validate the transformed document.
        #
        # Reference: https://github.com/microbiomedata/nmdc-schema/blob/main/src/docs/schema-validation.md
        #
        # Note: Dictionaries originating as Mongo documents include a Mongo-generated key named `_id`. However,
        #       the NMDC Schema does not describe that key and, indeed, data validators consider dictionaries
        #       containing that key to be invalid with respect to the NMDC Schema. So, here, we validate a
        #       copy (i.e. a shallow copy) of the document that lacks that specific key.
        #
        # Note: `root_to_validate` is a dictionary having the shape: { "some_collection_name": [ some_document ] }
        #       Reference: https://docs.python.org/3/library/stdtypes.html#dict (see the "type constructor" section)
        #
        document_without_underscore_id_key = {key: value for key, value in document.items() if key != "_id"}
        root_to_validate = dict([(collection_name, [document_without_underscore_id_key])])
        nmdc_jsonschema_validator.validate(root_to_validate)  # raises exception if invalid

### Dump the collections from the "transformer" MongoDB server

Now that the collections have been transformed and validated, dump them **from** the "transformer" MongoDB server **into** a local directory.

In [ ]:
# Dump the database from the "transformer" MongoDB server.
!{mongodump} \
  --config="{cfg.transformer_mongo_config_file_path}" \
  --db="nmdc" \
  --gzip \
  --out="{cfg.transformer_dump_folder_path}" \
  {exclusion_options_str}

### Create a bookkeeper

Create a `Bookkeeper` that can be used to document migration events in the "origin" server.

In [ ]:
bookkeeper = Bookkeeper(mongo_client=origin_mongo_client)

### Indicate — on the "origin" server — that the migration is underway

Add an entry to the migration log collection to indicate that this migration has started.

In [ ]:
bookkeeper.record_migration_event(migrator=migrator, event=MigrationEvent.MIGRATION_STARTED)

### TODO: Drop the original collections from the "origin" MongoDB server

This is necessary for situations where collections were renamed or deleted. The `--drop` option of `mongorestore` only drops collections that exist in the dump. We may need `mongosh` for this.

### Load the collections into the "origin" MongoDB server

Load the transformed collections into the "origin" MongoDB server, **replacing** the collections there that have the same names.

> Note: If the migration involved renaming or deleting a collection, the collection having the original name will continue to exist in the "origin" database until someone deletes it manually.

- Consider using the `--preserveUUID` CLI option

In [ ]:
# Replace the same-named collection(s) on the origin server, with the transformed one(s).
!{mongorestore} \
  --config="{cfg.origin_mongo_config_file_path}" \
  --gzip \
  --verbose \
  --dir="{cfg.transformer_dump_folder_path}" \
  --drop \
  --stopOnError \
  {inclusion_options_str}

### Indicate that the migration is complete

Add an entry to the migration log collection to indicate that this migration is complete.

In [ ]:
bookkeeper.record_migration_event(migrator=migrator, event=MigrationEvent.MIGRATION_COMPLETED)

### TODO: Reinstate write access to the MongoDB server

This effectively un-does the access revocation that we did earlier.